In [19]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [71]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [72]:
len(words)

32033

In [73]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(vocab_size)
print(itos)

27
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [74]:
# building the dataset
block_size = 3
def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [75]:
n_embed = 10
n_hidden = 200

In [76]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator=g, requires_grad=True)  # 27 chars, embedding dim 2
W1 = torch.randn((n_embed*block_size, n_hidden), generator=g, requires_grad=True)  # 6 chars, 100 hidden units
b1 = torch.randn(n_hidden, generator=g, requires_grad=True)  # 100 hidden units bias
W2 = torch.randn((n_hidden, vocab_size), generator=g, requires_grad=True)  # 100 hidden units, 27 chars
b2 = torch.randn(vocab_size, generator=g, requires_grad=True)  # 27 chars bias
parameters = [C, W1, b1, W2, b2]

In [77]:
batch_size = 32
max_steps = 200000
for i in range(max_steps):
    # construct a mini-batch of data
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))  # 32 random indices
    Xb, Yb = Xtr[ix], Ytr[ix]  # batch of data
    # forward pass
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1)  # flatten the batch of embeddings
    hpreact = embcat @ W1 + b1  # hidden layer pre-activation
    h = torch.tanh(hpreact)  # hidden layer activation
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Yb)

    # backward pass
    for p in parameters:
        p.grad = None  # zero the gradients

    # compute gradients
    loss.backward()

    lr = 0.1 if i < 100000 else 0.01  # learning rate schedule
    for p in parameters:
        p.data -= lr * p.grad  # gradient descent step
print(loss.item())

2.1544857025146484


In [79]:
@torch.no_grad()
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte)
    }[split]
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    h = torch.tanh(hpreact) 
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(f"{split} loss: {loss.item():.4f}")
split_loss('train')
split_loss('val')

train loss: 2.1206
val loss: 2.1664


In [80]:
# sample from the model
g = torch.Generator().manual_seed(2147483647+ 10)
for i in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(-1, n_embed*block_size) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

mona.
kayah.
see.
mad.
rylle.
emmastendrael.
azerydorliigh.
porelle.
elisananaraelynn.
hone.
miysadhvi.
shira.
stin.
joselynn.
nicolanu.
zen.
dariyah.
fieh.
yula.
myskeyla.
